# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located in the lab directory named `Sentiment140.csv.zip`. You need to unzip it into a `.csv` file. Then in the cell below, load and explore the data.

*Notes:* 

* The dataset was downloaded from [Kaggle](https://www.kaggle.com/kazanova/sentiment140). We made a slight change on the original data so that each column has a label.

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [11]:
# Import library
import pandas as pd
from nltk import FreqDist
import random

# Import local functions
from ipynb.fs.full.challenge_1 import clean_up
from ipynb.fs.full.challenge_1 import tokenize
from ipynb.fs.full.challenge_1 import stem_and_lemmatize
from ipynb.fs.full.challenge_1 import remove_stopwords

# Resource: https://stackoverflow.com/questions/44116194/import-a-function-from-another-ipynb-file

In [2]:
# Import dataset
data = pd.read_csv("../Sentiment140.csv")

In [3]:
# Take a random sample
# data = data.sample(n = 20000)
data = data.sample(n = 6000)

### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [4]:
# your code here
data["text_processed"] = data["text"].apply(clean_up)
data["text_processed"] = data["text_processed"].apply(tokenize)
data["text_processed"] = data["text_processed"].apply(stem_and_lemmatize)
data["text_processed"] = data["text_processed"].apply(remove_stopwords)
data.head(5)

,target,ids,date,flag,user,text,text_processed
434188,0,2065129067,Sun Jun 07 07:57:29 PDT 2009,NO_QUERY,ajkeeling,coursework done...now gotta try and magically ...,"[coursework, done, got, ta, tri, magic, remov,..."
1433792,4,2060511139,Sat Jun 06 19:02:32 PDT 2009,NO_QUERY,SneakySelector,lol i went to jiu jitsu class today and puked ...,"[lol, went, jiu, jitsu, class, today, puke, ti..."
876488,4,1681138816,Sat May 02 13:18:13 PDT 2009,NO_QUERY,ZachTocchi,@Lega_c you got it,"[lega_c, got]"
1385216,4,2052727779,Sat Jun 06 01:47:27 PDT 2009,NO_QUERY,caitieparker,@TragicScenery Thank you!! I really appreciat...,"[tragicsceneri, thank, realli, appreci, hope, ..."
1036124,4,1956293712,Thu May 28 21:34:24 PDT 2009,NO_QUERY,JaceyJanelle,"@thejohnset you should get on the live chat, i...","[thejohnset, get, live, chat, pretti, funni]"


In [5]:
data["target"].value_counts()

4    3039
0    2961
Name: target, dtype: int64

### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [6]:
# Create a list of all tweets with positive and negative
tweets = list(zip(data.text_processed, data.target))

# List of list of all words
list_of_tweets = data["text_processed"].tolist()

# Create one list of all words
all_words = []

for text in list_of_tweets:
    for t in text:
        all_words.append(t)

# Create frequency with words
all_words = FreqDist(all_words)

# Select the most common words
word_freq = all_words.most_common(5000)
word_freq

# List of list of words
bag = [[i for i,j in word_freq],
      [j for i,j in word_freq]]

# List of words
word_features = bag[0]

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [7]:
def find_features(document):
    words = set(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

In [8]:
featuresets = [(find_features(rev), category) for (rev, category) in tweets]
featuresets[0]

({'go': False,
  'day': False,
  'get': False,
  'wa': False,
  'good': False,
  'love': False,
  'like': False,
  'work': False,
  'quot': False,
  'u': False,
  'today': False,
  'time': False,
  'thank': False,
  'miss': False,
  'got': True,
  'amp': False,
  'lol': False,
  'back': False,
  'one': False,
  'see': False,
  'want': False,
  'feel': False,
  'know': False,
  'think': False,
  'make': True,
  'night': False,
  'im': False,
  'realli': False,
  'need': False,
  'last': False,
  'new': False,
  'ha': False,
  'hope': False,
  'watch': False,
  'come': False,
  'na': False,
  'great': False,
  'still': True,
  'well': False,
  'haha': False,
  'tomorrow': False,
  'home': False,
  'sorri': False,
  'look': False,
  'veri': False,
  'oh': False,
  'sad': False,
  'morn': False,
  'wish': False,
  'twitter': False,
  'follow': False,
  'sleep': False,
  'bad': False,
  'much': False,
  'friend': False,
  'tri': True,
  'whi': False,
  'right': False,
  'wait': False,
  'wo

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [12]:
# Import library
from nltk.classify import NaiveBayesClassifier

# Shuffle features
random.shuffle(featuresets)

# Split feature set into a training and a test with setnltk.NaiveBayesClassifier.train
train_set, test_set = featuresets[3000:], featuresets[:3000]
classifier = NaiveBayesClassifier.train(train_set)

# Inspect most important features with classifier.show_most_informative_features()
classifier.show_most_informative_features()


Most Informative Features
                    suck = True                0 : 4      =     10.7 : 1.0
                     sad = True                0 : 4      =     10.2 : 1.0
                   iphon = True                0 : 4      =     10.1 : 1.0
                     bad = True                0 : 4      =      9.6 : 1.0
                    wish = True                0 : 4      =      9.2 : 1.0
                  welcom = True                4 : 0      =      8.6 : 1.0
                    sick = True                0 : 4      =      8.5 : 1.0
                     soo = True                0 : 4      =      8.0 : 1.0
                   thank = True                4 : 0      =      8.0 : 1.0
                  pictur = True                4 : 0      =      7.3 : 1.0


### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [13]:
# Import library
import nltk.classify.util

# Calculate accuracy
nltk.classify.accuracy(classifier, test_set)

0.7023333333333334

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here